tmdb_5000_movies.csv : https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ai_school/data/tmdb_5000_movies.csv')
movies = movies.loc[:, ['title', 'genres', 'keywords']]

# 문자열로 되어있는 행을 인덱싱 할 수 있는 딕셔너리 형태로 변환
movies['genres'] = movies['genres'].apply(literal_eval)
movies['keywords'] = movies['keywords'].apply(literal_eval)

# i = 0
# for row in movies['genres']:
#     genres = []
#     for ele in row:
#         genres.append(ele['name'])
#     movies['genres'][i] = genres
#     i += 1

# 딕셔너리 형태를 깔끔하게 장르에 해당하는 부분만 뽑아서 문자열화
# [{}, {}, {}, {}] -> [장르, 장르, 장르, 장르]
movies['genres'] = movies['genres'].apply(lambda x : [y['name'] for y in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [y['name'] for y in x])

# [장르, 장르, 장르, 장르] -> 장르 장르 장르 장르
movies['genres'] = movies['genres'].apply(lambda x : ' '.join(x))
movies['keywords'] = movies['keywords'].apply(lambda x : ' '.join(x))

print(movies)

                                         title                                    genres                                           keywords
0                                       Avatar  Action Adventure Fantasy Science Fiction  culture clash future space war space colony so...
1     Pirates of the Caribbean: At World's End                  Adventure Fantasy Action  ocean drug abuse exotic island east india trad...
2                                      Spectre                    Action Adventure Crime  spy based on novel secret agent sequel mi6 bri...
3                        The Dark Knight Rises               Action Crime Drama Thriller  dc comics crime fighter terrorist secret ident...
4                                  John Carter          Action Adventure Science Fiction  based on novel mars medallion space travel pri...
...                                        ...                                       ...                                                ...
4798                

# TFIDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# ngram_range=(1, 2)는 단어를 1개 혹은 2개 연속으로 보겠다
# -> 단어 두 개를 이어 놓은 것도 한 단어라고 판단
# action adventure fantasy / adventure fantasy / adventure / fantasy
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(movies['genres'])
print(tfidf_vec.vocabulary_.items())
# 4803은 영화 갯수, 276은 tfidf 벡터가 가진 단어의 갯수 (장르의 종류)
# 하나의 영화를 276개의 열을 가진 벡터로 표현
print(tfidf_matrix.shape)

dict_items([('action', 0), ('adventure', 16), ('fantasy', 124), ('science', 232), ('fiction', 138), ('action adventure', 1), ('adventure fantasy', 24), ('fantasy science', 135), ('science fiction', 233), ('fantasy action', 125), ('crime', 64), ('adventure crime', 20), ('drama', 90), ('thriller', 234), ('action crime', 4), ('crime drama', 68), ('drama thriller', 106), ('adventure science', 29), ('animation', 33), ('family', 109), ('animation family', 38), ('fantasy family', 130), ('action science', 12), ('adventure action', 17), ('action thriller', 13), ('thriller crime', 238), ('western', 265), ('adventure western', 32), ('adventure family', 23), ('family fantasy', 115), ('fiction action', 139), ('action fantasy', 7), ('comedy', 44), ('action comedy', 3), ('comedy science', 59), ('adventure drama', 22), ('drama action', 91), ('romance', 214), ('drama romance', 104), ('romance thriller', 228), ('thriller action', 235), ('fiction thriller', 150), ('adventure thriller', 30), ('fantasy adv

In [4]:
# 유사도 행렬 (4803, 4803, 모든 영화들마다 다른 영화의 유사도가 하나의 행 안에 다 들어있음)
# 1, 1 (1번째 영화와 1번째 영화의 코사인 유사도)
# 1, 1 / 1, 2 / ... / 1, 4803 -> 1번째 영화와 1~4803번재 영화의 유사도
# 2, 1 / 2. 2 / ... / 2, 4803 -> 2번째 영화와 1~4803번째 영화의 유사도
# ...
# 4803, 1 / 4803, 2 / ... / 4803, 4803 -> 4803번째 영화와 1~4803번째 영화의 유사도
from sklearn.metrics.pairwise import cosine_similarity

# 4803개의 영화와 4803개의 영화간의 유사도를 구한다.
# 자기 자신과의 유사도는 무조건 1
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(genres_similarity)

# 유사도 값이 높은 영화의 제목
# 유사도 값이 높은 것부터 내림 차순으로 인덱스 값을 리턴 (유사도 값 자체가 아니라 유사도가 높은 영화가 뭔지 궁금하기 때문)
# n번째 행 : n번째 영화와 유사도가 높은 영화들의 인덱스가 유사도의 내림차순으로 정렬
similar_index = np.argsort(-genres_similarity) # -를 붙여서 내림차순으로 정렬
print(similar_index) # 1번째 행의 경우, 329가 앞에 나오는 이유는 장르가 같아 값이 동일하게 1이기 때문

[[1.         0.49309367 0.29270708 ... 0.         0.         0.        ]
 [0.49309367 1.         0.17786505 ... 0.         0.         0.        ]
 [0.29270708 0.17786505 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0  870 3494 ... 2355 2397 4802]
 [ 329  379  199 ... 2323 2325 4802]
 [1740    2 1542 ... 2359 2344 4802]
 ...
 [4800 3809 3285 ... 2085 2142 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 4593 4583 ... 1597 1595 2401]]


# Count (빈도 수)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(ngram_range=(1, 2))
count_matrix = count_vec.fit_transform(movies['keywords'])
print(count_matrix)

  (0, 9037)	1
  (0, 7010)	1
  (0, 15032)	1
  (0, 34485)	4
  (0, 40021)	2
  (0, 7433)	1
  (0, 34214)	1
  (0, 38236)	1
  (0, 15087)	1
  (0, 31412)	1
  (0, 1077)	2
  (0, 38408)	1
  (0, 27732)	1
  (0, 6161)	1
  (0, 22347)	1
  (0, 34275)	1
  (0, 3232)	1
  (0, 21496)	1
  (0, 548)	1
  (0, 1760)	1
  (0, 28328)	1
  (0, 30072)	1
  (0, 23178)	1
  (0, 1426)	1
  (0, 34431)	1
  :	:
  (4800, 41064)	1
  (4800, 19078)	1
  (4800, 9324)	1
  (4800, 2428)	1
  (4800, 24745)	1
  (4800, 33595)	1
  (4800, 21508)	1
  (4800, 2430)	1
  (4800, 14012)	1
  (4800, 9332)	1
  (4800, 28293)	1
  (4800, 33602)	1
  (4800, 24753)	1
  (4800, 19119)	1
  (4800, 36763)	1
  (4800, 28294)	1
  (4802, 8952)	1
  (4802, 25700)	1
  (4802, 11046)	1
  (4802, 15654)	1
  (4802, 5338)	1
  (4802, 11062)	1
  (4802, 25707)	1
  (4802, 5341)	1
  (4802, 8956)	1


In [6]:
# 유사도 행렬 (4803, 4803)
# 1, 1 (1번째 영화와 1번째 영화의 유사도)
# 1, 1 / 1, 2 / .... / 1, 4803
# 2, 1 / 2, 2 / .... / 2, 4803
# .....
# 4803, 1 / 4803, 2 / .... / 4803, 4803
from sklearn.metrics.pairwise import cosine_similarity

# 4803개의 영화랑 4803개의 영화끼리 유사도를 구하겠다!
genres_similarity = cosine_similarity(count_matrix, count_matrix)
print(genres_similarity)

# 유사도가 높은 영화를 알고 싶다!
# 유사도 값이 높은 것의 인덱스를 내림차순으로 출력/리턴
similar_index = np.argsort(-genres_similarity)
print(similar_index)

[[1.         0.01628008 0.         ... 0.02614435 0.         0.        ]
 [0.01628008 1.         0.         ... 0.03277368 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.02614435 0.03277368 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0 2403  278 ... 1996 1961 4802]
 [   1   12 2875 ... 1984 1971 4802]
 [   2   11  147 ... 1915 1880 4802]
 ...
 [4800 2674 2108 ... 1688 1697 4802]
 [   0 3205 3204 ... 1596 1594 4802]
 [4802 2000 4774 ... 1619 1635 2401]]


In [7]:
# 사용자가 입력한 영화의 인덱스 값을 찾아내고
# similar index 에 기록된 유사한 영화 인덱스를 찾아내고
# 유사한 영화 인덱스를 토대로 영화 이름을 찾아내면 된다!
input_movie = input()

movie_index = movies[movies['title'] == input_movie].index.values
print(movie_index)
similar_movies = similar_index[movie_index, :10]
print(similar_movies)
# 인덱스로 사용하기 위해서는 1차원으로 변형해줘야하기 때문에 reshape(-1)
similar_movies_index = similar_movies.reshape(-1)
print(similar_movies_index)
print(movies.iloc[similar_movies_index])

Begin Again
[3282]
[[   0 3205 3204 3203 3202 3201 3200 3206 3199 3197]]
[   0 3205 3204 3203 3202 3201 3200 3206 3199 3197]
                                         title                                    genres                                           keywords
0                                       Avatar  Action Adventure Fantasy Science Fiction  culture clash future space war space colony so...
3205                           Beneath Hill 60                         Drama History War                                                   
3204                              The I Inside          Thriller Mystery Science Fiction  infidelity amnesia paranoia nightmare time puz...
3203    Ultramarines: A Warhammer 40,000 Movie                 Animation Science Fiction                                                   
3202                          Crocodile Dundee                          Adventure Comedy  new york prostitute hotel journalist culture c...
3201                   Veronika Dec